# Ontology population 
In the following Jupyter notebook, we will populate our ontology using OwlReady2, a package for ontology-oriented programming in Python. It can load OWL 2.0 ontologies as Python objects, modify them, save them, and perform reasoning via HermiT.

In [1]:
! pip install owlready2

You should consider upgrading via the 'C:\Users\bordi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

First of all, we load our ontology. We loaded also the Dolce ontology since we used a class and a couple of properties belonging to it. This passage allows to use the correct prefix in both cases. 

In [3]:
from owlready2 import *
onto_path.append("C:/Users/bordi/OneDrive/Desktop/Gangemi/Ontologia")
onto = get_ontology("FinalUniOn.owl")
onto.load()


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


get_ontology("http://purl.org/ontology/UniOn#")

In [4]:
dul = get_ontology("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl")
dul.load()

get_ontology("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")

We try to print our classes to check everything is fine and loaded correctly. 

In [5]:
print(list(onto.classes()))

[FinalUniOn.DegreeProgramme, FinalUniOn.ItalianPublicUniversity, FinalUniOn.AcademicField, FinalUniOn.AccessType, FinalUniOn.AssessmentMethod, FinalUniOn.EducationalActivity, DUL.Place, FinalUniOn.Curriculum, FinalUniOn.Department, FinalUniOn.Qualification, FinalUniOn.BachelorDegree, FinalUniOn.Course, FinalUniOn.FinalTest, FinalUniOn.InternationalDegree, FinalUniOn.Internship, FinalUniOn.Laboratory, FinalUniOn.LimitedAccessDegree, FinalUniOn.OpenAccessDegree, FinalUniOn.MasterDegree, FinalUniOn.Seminar, FinalUniOn.SingleCycleDegree]


We decided to treat our final data frame as a dictionary to better iterate on its values and popolate our ontology. 

In [6]:
data = pd.read_csv("FinalData.csv", sep=",", encoding="utf-8")

In [7]:
data = data.to_dict("index")

To populate our classes we need to iterate on our values, creating the individuals, their labels, and establishing relationships among them. 

In [8]:
#we create all the Degree programmes, keeping as the IRI the final entity number.
#then, we add to the international subclass the degree programmes declared as international in our df. 

for record in data.values():
    my_degree = onto.DegreeProgramme(record["DegreeProgramme"].split("/")[-1])
    if record["International"] is True:
        my_degree = onto.InternationalDegree(record["DegreeProgramme"].split("/")[-1])
        
        
    #differentiate degree programmes based on types       
    if record["type"] == "Laurea":
        my_degree = onto.BachelorDegree(record["DegreeProgramme"].split("/")[-1])
    if record["type"] == "Laurea Magistrale":
        my_degree = onto.MasterDegree(record["DegreeProgramme"].split("/")[-1])
    if record["type"] == "Laurea Magistrale a ciclo unico":
        my_degree = onto.SingleCycleDegree(record["DegreeProgramme"].split("/")[-1])
        
        
    #differentiate degree programmes based on access        
    if "libero accesso" in record["AccessType"]: 
        my_degree.hasAccessType = [onto.AccessType("OpenAccess")]
        my_degree = onto.OpenAccessDegree(my_degree)
               
    if "numero chiuso" in record["AccessType"]:
        my_degree.hasAccessType = [onto.AccessType("LimitedAccess")]
        my_degree = onto.LimitedAccessDegree(my_degree)


    #giving a label to all Degree programmes                
    my_degree.label = record["DegreeProgrammeLabel"]

    #academic field
    my_academicField = onto.AcademicField(record["AcademicField"].replace(" ", "_"))
    my_academicField.label = record["AcademicField"]
    
    my_degree.hasAcademicField.append(my_academicField)
    
    #italian university 
    my_university = onto.ItalianPublicUniversity(record["Italian University"].replace(" ", "_"))
    my_university.label = record["Italian University"]
    
    my_degree.isDeliveredBy = [onto.ItalianPublicUniversity(my_university)]
    
    #type of educational activities and identifiers 
    if "Exam" in record["typeOfActivity"]: 
        my_educational_activity = onto.Course(record["EducationalActivity"].replace(" ", "_"))
        my_educational_activity.hasSsd = [record["Ssd"]]
        
    if "Internship" in record["typeOfActivity"]: 
        my_educational_activity = onto.Internship(record["EducationalActivity"].replace(" ", "_"))
    if "Seminar" in record["typeOfActivity"]: 
        my_educational_activity = onto.Seminar(record["EducationalActivity"].replace(" ", "_"))
    if "Laboratory" in record["typeOfActivity"]: 
        my_educational_activity = onto.Laboratory(record["EducationalActivity"].replace(" ", "_"))
    if "Final Test" in record["typeOfActivity"]: 
        my_educational_activity = onto.FinalTest(record["EducationalActivity"].replace(" ", "_"))
    
          
    my_educational_activity.label = [record["EducationalActivity"]]
    my_educational_activity.hasCfu = [record["Cfu"]]
    my_educational_activity.hasTeachingCode = [record["TeachingCode"]]
    
    
    my_degree.hasEducationalActivity.append(my_educational_activity)
    
    
    #departments and relationship between Degree programmes - department, university - department
    
    my_department = onto.Department(record["Department"].replace(" ", "_"))
    my_department.label = record["Department"]
    
    my_degree.isManagedBy = [onto.Department(my_department)]

    #relazione tra dipartimenti e università 
    
    my_department.isPartOf.append(my_university)


    #url for degree programmes
    my_degree.hasUrl = [record["DegreeProgramme"]] #data property    
    
    #academic year    
    my_degree.hasAcademicYear = [record["academicYear"]] #data property
    
    #degree class    
    my_degree.hasDegreeClass = [record["DegreeClass"]] #data property 

    #yearsofDuration
    my_degree.hasYearsOfDuration = [record["hasYearsOfDuration"]] #data property 
    
    #language
    my_degree.hasTeachingLanguage = [record["TeachingLanguage"]] #data property 
   

    #qualification 
    if "Titolo unico" in record["Qualification"]: 
        my_degree.releases = [onto.Qualification("SingleQualification")]               
    if "Titolo multiplo" in record["Qualification"]:
        my_degree.releases = [onto.Qualification("MultipleQualification")]
    if "Titolo congiunto" in record["Qualification"]:
        my_degree.releases = [onto.Qualification("JointQualification")]  

    #places (campus)
    my_campus = dul.Place(record["campus"].replace(" ", "_"))
    my_campus.label = record["campus"]
                               
    my_degree.hasCampus = [dul.Place(my_campus)]
                                                           
    #places (placesOfTeaching)
    my_teachingPlace = dul.Place(record["PlaceOfTeaching"].replace(" ", "_"))
    my_teachingPlace.label = record["PlaceOfTeaching"]
    
    my_degree.hasPlaceOfTeaching = [dul.Place(my_teachingPlace)]


In [9]:
onto.save("UniOn.owl")

Once the Reasoner is started, it deduces correctly what has been not declared manually. Thus, we can say our ontology seems correctly structured and working. 